In [59]:
from bs4 import BeautifulSoup
import urllib.request,sys,time
import requests
import pandas as pd
import re
import string
import numpy as np
import nltk
from math import log10
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ahish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [60]:
#url of the page that we want to Scarpe
#+str() is used to convert int datatype of the page no. and concatenate that to a URL for pagination purposes.
page = 1
url = 'https://www.politifact.com/factchecks/list/?page='+str(page)

#Use the browser to get the URL. This is a suspicious command that might blow up.

In [61]:
try:
     # this might throw an exception if something goes wrong.
        page=requests.get(url)      
    # this describes what to do if an exception is thrown 
except Exception as e:
    error_type, error_obj, error_info = sys.exc_info()      
    
    #print the link that cause the problem
    print ('ERROR FOR LINK:',url)
    
    #print error info and line that threw the exception                          
    print (error_type, 'Line:', error_info.tb_lineno)
    

In [62]:
page.status_code

200

In [63]:
if "Politifact" in page.text:
    print("Yes, Scarpe it")

print (page.headers.get("content-type", "unknown"))

Yes, Scarpe it
text/html; charset=utf-8


In [65]:
# Main logic

#Total number of pages of the websites to crawl
pagesToGet = 1

frame = []
upperframe = []

for pageNo in range(1,pagesToGet+1):
    
    url = 'https://www.politifact.com/factchecks/list/?page='+str(pageNo)+'&ruling=true'
    
    try:
        page=requests.get(url)      
    except Exception as e:
        error_type, error_obj, error_info = sys.exc_info()      

        print ('ERROR FOR LINK:',url)

        print (error_type, 'Line:', error_info.tb_lineno)
        continue
    
#     time.sleep(2)  
    
    soup = BeautifulSoup(page.text, "html.parser")
    links=soup.find_all('li',attrs={'class':'o-listicle__item'})

    print("Processing page "+str(pageNo)+"...")

    #Segregating the contents found in the article
    for j in links:
#         Statement = j.find("div",attrs={'class':'m-statement__quote'}).text.strip()
        Link = "https://www.politifact.com"
        Link += j.find("div",attrs={'class':'m-statement__quote'}).find('a')['href'].strip()
        
        try:
            page=requests.get(Link)      
        except Exception as e:
            print(e)
        
        soup = BeautifulSoup(page.text, "html.parser")
        Title=soup.find('h2',attrs={'class':'c-title c-title--subline'}).text.strip()
        ArticleTemp=soup.find('article',attrs={'class':'m-textblock'})
        Article = soup.find_all('p')
        fullArticle = []
        for i in range(2,len(Article)):
            fullArticle.append(Article[i].text.strip())
        fullArticle = " ".join(fullArticle)
        
        Date = j.find('div',attrs={'class':'m-statement__body'}).find('footer').text.split("•")[1]
        Date = Date.replace("\n", "")
        Source = j.find('div', attrs={'class':'m-statement__meta'}).find('a').text.strip()
#         Label = j.find('div', attrs ={'class':'m-statement__content'}).find('img',attrs={'class':'c-image__original'}).get('alt').strip()
        frame.append([Title, fullArticle, Link,Date,Source])
    print("Done with page "+str(pageNo)+"!")
    upperframe.extend(frame)

Processing page 1...
Done with page 1!


In [88]:
data=pd.DataFrame(upperframe, columns=['Title', 'Article', 'Link','Date','Source'])
data.head()

,Title,Article,Link,Date,Source
0,COVID-19 fatalities in the Rio Grande Valley d...,Health workers prepare to administer a COVID-1...,https://www.politifact.com/factchecks/2020/oct...,"October 16, 2020",Vicente Gonzalez
1,Roy Cooper compares Georgia's COVID-19 numbers...,North Carolina Gov. Roy Cooper speaks during a...,https://www.politifact.com/factchecks/2020/oct...,"October 15, 2020",Roy Cooper
2,Fact-checking Jim Justice on West Virginia’s v...,West Virginia Gov. Jim Justice prepares for a ...,https://www.politifact.com/factchecks/2020/oct...,"October 15, 2020",Jim Justice
3,"Yes, Wisconsin legislators haven’t passed a bi...",The Wisconsin Legislature last passed a bill i...,https://www.politifact.com/factchecks/2020/oct...,"October 7, 2020",Facebook posts
4,FBI director warned about white supremacist vi...,Democratic presidential candidate former Vice ...,https://www.politifact.com/factchecks/2020/oct...,"October 6, 2020",Joe Biden


In [89]:
len(data)

30

In [90]:
def stemming(token):
  if (token in set(nltk.corpus.stopwords.words('english'))):
    return token
  return nltk.stem.PorterStemmer().stem(token)

In [91]:
def text_tokenizer(text):
  tokens = nltk.word_tokenize(text)
  stemmed_tokens = (stemming(token) for token in tokens) #stemming and removing stop words
  return list([token for token in stemmed_tokens if token.isalnum()]) #removes tokens that are neither alphabetic characters nor digits

In [92]:
corpus = data["Article"].tolist()

In [93]:
count_vector = CountVectorizer(stop_words = nltk.word_tokenize(' '.join(nltk.corpus.stopwords.words('english'))), tokenizer = text_tokenizer)
tokenized_documents = count_vector.fit_transform(corpus)
term_document_matrix = pd.DataFrame(tokenized_documents.toarray(), columns = count_vector.get_feature_names())

In [94]:
cols = sorted(list(term_document_matrix.columns))
data = []
for col in cols:
  l = term_document_matrix[col].loc[term_document_matrix[col]>0].index.to_list()
  data.append([col, len(l), l])
inverted_index = pd.DataFrame(data ,columns=["Term","Doc_freq","Postings_List"])
print("Inverted index\n",inverted_index)

Inverted index
            Term  Doc_freq                                      Postings_List
0            07         1                                               [11]
1             1        13     [2, 3, 4, 5, 7, 9, 11, 15, 16, 18, 23, 24, 26]
2            10        14  [2, 3, 6, 8, 9, 10, 12, 13, 15, 17, 18, 23, 24...
3           100         5                                [5, 16, 18, 19, 28]
4          1038         1                                                [3]
...         ...       ...                                                ...
2767    younger         1                                                [6]
2768      youth         1                                               [29]
2769     youtub         4                                   [22, 23, 28, 29]
2770       zero         1                                               [19]
2771  zuckerman         1                                                [5]

[2772 rows x 3 columns]
